In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from keras.layers import Dense, Dropout
from keras.models import Model, Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv("lebron_dataset.csv",index_col=0)
df.head()

,G,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,BLK,TOV,PF,PTS,GmSc,"""+/-""",Win,Winby,minutes,age_y
0,1,12,20,0.600,0,2,0.0,1,3,0.333,...,0,2,3,25,24.7,-9,0,-14,42.833333,18.830137
1,2,8,17,0.471,1,5,0.2,4,7,0.571,...,0,7,1,21,14.7,-3,0,-9,40.350000,18.832877
2,3,3,12,0.250,0,1,0.0,2,2,1.000,...,0,2,3,8,5.0,-21,0,-19,39.166667,18.838356
3,4,3,11,0.273,0,2,0.0,1,1,1.000,...,3,2,1,7,11.2,-3,0,-4,41.100000,18.849315
4,5,8,18,0.444,1,2,0.5,6,7,0.857,...,0,7,2,23,9.0,-7,0,-1,43.733333,18.854795


In [3]:
df = df.astype('float32')

In [4]:
y = df['Win']
X = df.drop(columns=['Win','Winby'])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

In [6]:
def my_DL(epochs=6,batchsize=512):
    model = Sequential()
    model.add(Dense(32,activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
    return model

In [7]:
def train_hyper_tune(X,y):
    # create the pre-processing component
    my_scaler = StandardScaler()
    my_imputer = SimpleImputer(strategy="median")
    
    # define classifiers
    ## Classifier 1: Logistic Regression
    clf_LR = LogisticRegression(random_state=0,penalty='elasticnet',solver='saga')
    ## Classifier 2: Random Forest Classifier
    clf_RF = RandomForestClassifier(random_state=0)
    ## Classifier 3: Deep Learning Binary Classifier
    clf_DL = KerasClassifier(build_fn=my_DL)
    
    # define pipeline for three classifiers
    ## clf_LR
    pipe1 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('lr_model',clf_LR)])
    ## clf_RF
    pipe2 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('rf_model',clf_RF)])
    ## clf_DL
    pipe3 = Pipeline([('imputer', my_imputer), ('scaler', my_scaler), ('dl_model',clf_DL)])
    
    # create hyperparameter space of the three models
    ## clf_LR
    param_grid1 = {
        'lr_model__C' : [1e-1,1,10],
        'lr_model__l1_ratio' : [0,0.5,1]
    }
    ## clf_RF
    param_grid2 = {
        'rf_model__n_estimators' : [50,100],
        'rf_model__max_features' : [0.8,"auto"],
        'rf_model__max_depth' : [4,5]
    }
    ## clf_DL
    param_grid3 = {
        'dl_model__epochs' : [6,12,18,24],
        'dl_model__batchsize' : [256,512]
    }
    
    # set GridSearch via 5-fold cross-validation
    ## clf_LR
    grid1 = GridSearchCV(pipe1, cv=5, param_grid=param_grid1)
    ## clf_RF
    grid2 = GridSearchCV(pipe2, cv=5, param_grid=param_grid2)
    ## clf_DL
    grid3 = GridSearchCV(pipe3, cv=5, param_grid=param_grid3)
    
    # run the hyperparameter tunning process
    grid1.fit(X,y)
    grid2.fit(X,y)
    grid3.fit(X,y)
    
    # return results of the tunning process
    return grid1,grid2,grid3,pipe1,pipe2,pipe3

In [8]:
my_grid1,my_grid2,my_grid3,my_pipe1,my_pipe2,my_pipe3 = train_hyper_tune(X_train, y_train)

/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sa

Epoch 1/6
31/31 [==============================] - 0s 2ms/step - loss: 0.6090 - accuracy: 0.6670
Epoch 2/6
31/31 [==============================] - 0s 1ms/step - loss: 0.5295 - accuracy: 0.7295
Epoch 3/6
31/31 [==============================] - 0s 1ms/step - loss: 0.4764 - accuracy: 0.7879
Epoch 4/6
31/31 [==============================] - 0s 1ms/step - loss: 0.4318 - accuracy: 0.8207
Epoch 5/6
31/31 [==============================] - 0s 1ms/step - loss: 0.3938 - accuracy: 0.8299
Epoch 6/6
8/8 [==============================] - 0s 1ms/step - loss: 0.3132 - accuracy: 0.8811
Epoch 1/6
31/31 [==============================] - 0s 1ms/step - loss: 0.5970 - accuracy: 0.6680
Epoch 2/6
31/31 [==============================] - 0s 1ms/step - loss: 0.4861 - accuracy: 0.7797
Epoch 3/6
31/31 [==============================] - 0s 2ms/step - loss: 0.4206 - accuracy: 0.8207
Epoch 4/6
31/31 [==============================] - 0s 2ms/step - loss: 0.3768 - accuracy: 0.8391
Epoch 5/6
31/31 [===============

31/31 [==============================] - 0s 2ms/step - loss: 0.2471 - accuracy: 0.8996
Epoch 12/12
8/8 [==============================] - 0s 1ms/step - loss: 0.3347 - accuracy: 0.8607
Epoch 1/12
31/31 [==============================] - 0s 933us/step - loss: 0.6045 - accuracy: 0.6967
Epoch 2/12
31/31 [==============================] - 0s 1ms/step - loss: 0.5031 - accuracy: 0.7684
Epoch 3/12
31/31 [==============================] - 0s 974us/step - loss: 0.4304 - accuracy: 0.8115
Epoch 4/12
31/31 [==============================] - 0s 1ms/step - loss: 0.3776 - accuracy: 0.8320
Epoch 5/12
31/31 [==============================] - 0s 1ms/step - loss: 0.3392 - accuracy: 0.8525
Epoch 6/12
31/31 [==============================] - 0s 1ms/step - loss: 0.3112 - accuracy: 0.8658
Epoch 7/12
31/31 [==============================] - 0s 1ms/step - loss: 0.2891 - accuracy: 0.8770
Epoch 8/12
31/31 [==============================] - 0s 1ms/step - loss: 0.2713 - accuracy: 0.8863
Epoch 9/12
31/31 [==========

31/31 [==============================] - 0s 954us/step - loss: 0.2609 - accuracy: 0.8965
Epoch 11/18
31/31 [==============================] - 0s 1ms/step - loss: 0.2512 - accuracy: 0.9047
Epoch 12/18
31/31 [==============================] - 0s 992us/step - loss: 0.2441 - accuracy: 0.9027
Epoch 13/18
31/31 [==============================] - 0s 967us/step - loss: 0.2376 - accuracy: 0.9068
Epoch 14/18
31/31 [==============================] - 0s 979us/step - loss: 0.2320 - accuracy: 0.9068
Epoch 15/18
31/31 [==============================] - 0s 1ms/step - loss: 0.2272 - accuracy: 0.9109
Epoch 16/18
31/31 [==============================] - 0s 898us/step - loss: 0.2229 - accuracy: 0.9150
Epoch 17/18
31/31 [==============================] - 0s 941us/step - loss: 0.2170 - accuracy: 0.9170
Epoch 18/18
8/8 [==============================] - 0s 687us/step - loss: 0.2970 - accuracy: 0.8811
Epoch 1/18
31/31 [==============================] - 0s 795us/step - loss: 0.6591 - accuracy: 0.6178
Epoch 2/1

31/31 [==============================] - 0s 1ms/step - loss: 0.3664 - accuracy: 0.8555
Epoch 5/24
31/31 [==============================] - 0s 1ms/step - loss: 0.3290 - accuracy: 0.8678
Epoch 6/24
31/31 [==============================] - 0s 1ms/step - loss: 0.3006 - accuracy: 0.8873
Epoch 7/24
31/31 [==============================] - 0s 927us/step - loss: 0.2793 - accuracy: 0.8945
Epoch 8/24
31/31 [==============================] - 0s 921us/step - loss: 0.2641 - accuracy: 0.8986
Epoch 9/24
31/31 [==============================] - 0s 956us/step - loss: 0.2521 - accuracy: 0.9016
Epoch 10/24
31/31 [==============================] - 0s 958us/step - loss: 0.2420 - accuracy: 0.9016
Epoch 11/24
31/31 [==============================] - 0s 1ms/step - loss: 0.2350 - accuracy: 0.9037
Epoch 12/24
31/31 [==============================] - 0s 1ms/step - loss: 0.2292 - accuracy: 0.9119
Epoch 13/24
31/31 [==============================] - 0s 945us/step - loss: 0.2240 - accuracy: 0.9098
Epoch 14/24
31/31

Epoch 5/6
31/31 [==============================] - 0s 5ms/step - loss: 0.4039 - accuracy: 0.8279
Epoch 6/6
8/8 [==============================] - 0s 2ms/step - loss: 0.3387 - accuracy: 0.8648
Epoch 1/6
31/31 [==============================] - 0s 884us/step - loss: 0.6265 - accuracy: 0.6629
Epoch 2/6
31/31 [==============================] - 0s 954us/step - loss: 0.5314 - accuracy: 0.7592
Epoch 3/6
31/31 [==============================] - 0s 1ms/step - loss: 0.4647 - accuracy: 0.7941
Epoch 4/6
31/31 [==============================] - 0s 1ms/step - loss: 0.4103 - accuracy: 0.8217
Epoch 5/6
31/31 [==============================] - 0s 1ms/step - loss: 0.3673 - accuracy: 0.8412
Epoch 6/6
8/8 [==============================] - 0s 712us/step - loss: 0.3602 - accuracy: 0.8279
Epoch 1/6
31/31 [==============================] - 0s 861us/step - loss: 0.5428 - accuracy: 0.7203
Epoch 2/6
31/31 [==============================] - 0s 947us/step - loss: 0.4634 - accuracy: 0.7982
Epoch 3/6
31/31 [=======

31/31 [==============================] - 0s 1ms/step - loss: 0.2675 - accuracy: 0.8893
Epoch 9/12
31/31 [==============================] - 0s 1ms/step - loss: 0.2509 - accuracy: 0.8996
Epoch 10/12
31/31 [==============================] - 0s 1ms/step - loss: 0.2388 - accuracy: 0.9109
Epoch 11/12
31/31 [==============================] - 0s 982us/step - loss: 0.2292 - accuracy: 0.9119
Epoch 12/12
8/8 [==============================] - 0s 798us/step - loss: 0.3251 - accuracy: 0.8770
Epoch 1/18
31/31 [==============================] - 0s 885us/step - loss: 0.6889 - accuracy: 0.5707
Epoch 2/18
31/31 [==============================] - 0s 948us/step - loss: 0.5891 - accuracy: 0.6998
Epoch 3/18
31/31 [==============================] - 0s 959us/step - loss: 0.5129 - accuracy: 0.7602
Epoch 4/18
31/31 [==============================] - 0s 982us/step - loss: 0.4482 - accuracy: 0.8084
Epoch 5/18
31/31 [==============================] - 0s 979us/step - loss: 0.4002 - accuracy: 0.8279
Epoch 6/18
31/31

31/31 [==============================] - 0s 831us/step - loss: 0.5484 - accuracy: 0.7264
Epoch 3/18
31/31 [==============================] - 0s 985us/step - loss: 0.4919 - accuracy: 0.7684
Epoch 4/18
31/31 [==============================] - 0s 1ms/step - loss: 0.4448 - accuracy: 0.7951
Epoch 5/18
31/31 [==============================] - 0s 941us/step - loss: 0.4042 - accuracy: 0.8186
Epoch 6/18
31/31 [==============================] - 0s 909us/step - loss: 0.3691 - accuracy: 0.8422
Epoch 7/18
31/31 [==============================] - 0s 885us/step - loss: 0.3377 - accuracy: 0.8586
Epoch 8/18
31/31 [==============================] - 0s 877us/step - loss: 0.3119 - accuracy: 0.8678
Epoch 9/18
31/31 [==============================] - 0s 903us/step - loss: 0.2917 - accuracy: 0.8801
Epoch 10/18
31/31 [==============================] - 0s 891us/step - loss: 0.2747 - accuracy: 0.8883
Epoch 11/18
31/31 [==============================] - 0s 890us/step - loss: 0.2603 - accuracy: 0.8986
Epoch 12/18

31/31 [==============================] - 0s 947us/step - loss: 0.2106 - accuracy: 0.9191
Epoch 16/24
31/31 [==============================] - 0s 1ms/step - loss: 0.2069 - accuracy: 0.9211
Epoch 17/24
31/31 [==============================] - 0s 1ms/step - loss: 0.2014 - accuracy: 0.9262
Epoch 18/24
31/31 [==============================] - 0s 957us/step - loss: 0.1979 - accuracy: 0.9262
Epoch 19/24
31/31 [==============================] - 0s 1ms/step - loss: 0.1927 - accuracy: 0.9252
Epoch 20/24
31/31 [==============================] - 0s 979us/step - loss: 0.1919 - accuracy: 0.9283
Epoch 21/24
31/31 [==============================] - 0s 1ms/step - loss: 0.1863 - accuracy: 0.9262
Epoch 22/24
31/31 [==============================] - 0s 990us/step - loss: 0.1837 - accuracy: 0.9273
Epoch 23/24
31/31 [==============================] - 0s 988us/step - loss: 0.1808 - accuracy: 0.9293
Epoch 24/24
8/8 [==============================] - 0s 897us/step - loss: 0.3156 - accuracy: 0.8811
Epoch 1/24
3

39/39 [==============================] - 0s 894us/step - loss: 0.2090 - accuracy: 0.9139
Epoch 23/24
39/39 [==============================] - 0s 1ms/step - loss: 0.2080 - accuracy: 0.9107
Epoch 24/24
39/39 [==============================] - 0s 891us/step - loss: 0.2041 - accuracy: 0.9156


In [9]:
print(train_hyper_tune(X_train, y_train))

/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/rushipatel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sa

Epoch 1/6
31/31 [==============================] - 0s 1ms/step - loss: 0.6425 - accuracy: 0.6557
Epoch 2/6
31/31 [==============================] - 0s 2ms/step - loss: 0.5560 - accuracy: 0.7582
Epoch 3/6
31/31 [==============================] - 0s 1ms/step - loss: 0.4933 - accuracy: 0.7848
Epoch 4/6
31/31 [==============================] - 0s 1ms/step - loss: 0.4399 - accuracy: 0.8176
Epoch 5/6
31/31 [==============================] - 0s 1ms/step - loss: 0.3977 - accuracy: 0.8361
Epoch 6/6
8/8 [==============================] - 0s 708us/step - loss: 0.3085 - accuracy: 0.8852
Epoch 1/6
31/31 [==============================] - 0s 716us/step - loss: 0.6183 - accuracy: 0.6844
Epoch 2/6
31/31 [==============================] - 0s 1ms/step - loss: 0.5239 - accuracy: 0.7490
Epoch 3/6
31/31 [==============================] - 0s 2ms/step - loss: 0.4572 - accuracy: 0.8012
Epoch 4/6
31/31 [==============================] - 0s 1ms/step - loss: 0.4049 - accuracy: 0.8340
Epoch 5/6
31/31 [===========

31/31 [==============================] - 0s 903us/step - loss: 0.2559 - accuracy: 0.8934
Epoch 12/12
8/8 [==============================] - 0s 5ms/step - loss: 0.3051 - accuracy: 0.8852
Epoch 1/12
31/31 [==============================] - 0s 941us/step - loss: 0.6218 - accuracy: 0.6609
Epoch 2/12
31/31 [==============================] - 0s 2ms/step - loss: 0.5255 - accuracy: 0.7449
Epoch 3/12
31/31 [==============================] - 0s 1ms/step - loss: 0.4572 - accuracy: 0.7910
Epoch 4/12
31/31 [==============================] - 0s 979us/step - loss: 0.4065 - accuracy: 0.8207
Epoch 5/12
31/31 [==============================] - 0s 1ms/step - loss: 0.3694 - accuracy: 0.8391
Epoch 6/12
31/31 [==============================] - 0s 910us/step - loss: 0.3383 - accuracy: 0.8443
Epoch 7/12
31/31 [==============================] - 0s 1ms/step - loss: 0.3135 - accuracy: 0.8658
Epoch 8/12
31/31 [==============================] - 0s 943us/step - loss: 0.2914 - accuracy: 0.8873
Epoch 9/12
31/31 [====

Epoch 11/18
31/31 [==============================] - 0s 1ms/step - loss: 0.2319 - accuracy: 0.9160
Epoch 12/18
31/31 [==============================] - 0s 1ms/step - loss: 0.2247 - accuracy: 0.9180
Epoch 13/18
31/31 [==============================] - 0s 934us/step - loss: 0.2175 - accuracy: 0.9221
Epoch 14/18
31/31 [==============================] - 0s 944us/step - loss: 0.2126 - accuracy: 0.9273
Epoch 15/18
31/31 [==============================] - 0s 1ms/step - loss: 0.2076 - accuracy: 0.9242
Epoch 16/18
31/31 [==============================] - 0s 1ms/step - loss: 0.2030 - accuracy: 0.9262
Epoch 17/18
31/31 [==============================] - 0s 947us/step - loss: 0.1986 - accuracy: 0.9283
Epoch 18/18
8/8 [==============================] - 0s 678us/step - loss: 0.3039 - accuracy: 0.8893
Epoch 1/18
31/31 [==============================] - 0s 895us/step - loss: 0.6344 - accuracy: 0.6547
Epoch 2/18
31/31 [==============================] - 0s 1ms/step - loss: 0.5257 - accuracy: 0.7602
Epoc

31/31 [==============================] - 0s 882us/step - loss: 0.3811 - accuracy: 0.8412
Epoch 6/24
31/31 [==============================] - 0s 958us/step - loss: 0.3417 - accuracy: 0.8555
Epoch 7/24
31/31 [==============================] - 0s 869us/step - loss: 0.3115 - accuracy: 0.8699
Epoch 8/24
31/31 [==============================] - 0s 898us/step - loss: 0.2883 - accuracy: 0.8811
Epoch 9/24
31/31 [==============================] - 0s 904us/step - loss: 0.2708 - accuracy: 0.8924
Epoch 10/24
31/31 [==============================] - 0s 889us/step - loss: 0.2556 - accuracy: 0.9006
Epoch 11/24
31/31 [==============================] - 0s 885us/step - loss: 0.2454 - accuracy: 0.9006
Epoch 12/24
31/31 [==============================] - 0s 879us/step - loss: 0.2349 - accuracy: 0.9068
Epoch 13/24
31/31 [==============================] - 0s 831us/step - loss: 0.2278 - accuracy: 0.9129
Epoch 14/24
31/31 [==============================] - 0s 883us/step - loss: 0.2224 - accuracy: 0.9109
Epoch 

31/31 [==============================] - 0s 1ms/step - loss: 0.4107 - accuracy: 0.8115
Epoch 6/6
8/8 [==============================] - 0s 794us/step - loss: 0.3467 - accuracy: 0.8361
Epoch 1/6
31/31 [==============================] - 0s 855us/step - loss: 0.5973 - accuracy: 0.6824
Epoch 2/6
31/31 [==============================] - 0s 864us/step - loss: 0.5228 - accuracy: 0.7408
Epoch 3/6
31/31 [==============================] - 0s 971us/step - loss: 0.4621 - accuracy: 0.7941
Epoch 4/6
31/31 [==============================] - 0s 979us/step - loss: 0.4110 - accuracy: 0.8197
Epoch 5/6
31/31 [==============================] - 0s 855us/step - loss: 0.3708 - accuracy: 0.8361
Epoch 6/6
8/8 [==============================] - 0s 694us/step - loss: 0.3422 - accuracy: 0.8361
Epoch 1/6
31/31 [==============================] - 0s 850us/step - loss: 0.6378 - accuracy: 0.6773
Epoch 2/6
31/31 [==============================] - 0s 991us/step - loss: 0.5483 - accuracy: 0.7090
Epoch 3/6
31/31 [=========

31/31 [==============================] - 0s 1ms/step - loss: 0.2906 - accuracy: 0.8781
Epoch 9/12
31/31 [==============================] - 0s 1ms/step - loss: 0.2729 - accuracy: 0.8893
Epoch 10/12
31/31 [==============================] - 0s 1ms/step - loss: 0.2587 - accuracy: 0.8996
Epoch 11/12
31/31 [==============================] - 0s 996us/step - loss: 0.2472 - accuracy: 0.9068
Epoch 12/12
8/8 [==============================] - 0s 991us/step - loss: 0.3285 - accuracy: 0.8689
Epoch 1/18
31/31 [==============================] - 0s 951us/step - loss: 0.7501 - accuracy: 0.4477
Epoch 2/18
31/31 [==============================] - 0s 1ms/step - loss: 0.6126 - accuracy: 0.6906
Epoch 3/18
31/31 [==============================] - 0s 1ms/step - loss: 0.5346 - accuracy: 0.7633
Epoch 4/18
31/31 [==============================] - 0s 1ms/step - loss: 0.4753 - accuracy: 0.7869
Epoch 5/18
31/31 [==============================] - 0s 1ms/step - loss: 0.4248 - accuracy: 0.8207
Epoch 6/18
31/31 [======

31/31 [==============================] - 0s 901us/step - loss: 0.5402 - accuracy: 0.7705
Epoch 3/18
31/31 [==============================] - 0s 1ms/step - loss: 0.4664 - accuracy: 0.8033
Epoch 4/18
31/31 [==============================] - 0s 997us/step - loss: 0.4043 - accuracy: 0.8340
Epoch 5/18
31/31 [==============================] - 0s 977us/step - loss: 0.3562 - accuracy: 0.8535
Epoch 6/18
31/31 [==============================] - 0s 1ms/step - loss: 0.3200 - accuracy: 0.8832
Epoch 7/18
31/31 [==============================] - 0s 1ms/step - loss: 0.2934 - accuracy: 0.8863
Epoch 8/18
31/31 [==============================] - 0s 1ms/step - loss: 0.2727 - accuracy: 0.8904
Epoch 9/18
31/31 [==============================] - 0s 1ms/step - loss: 0.2575 - accuracy: 0.8986
Epoch 10/18
31/31 [==============================] - 0s 1ms/step - loss: 0.2468 - accuracy: 0.8996
Epoch 11/18
31/31 [==============================] - 0s 1ms/step - loss: 0.2363 - accuracy: 0.9098
Epoch 12/18
31/31 [====

31/31 [==============================] - 0s 1ms/step - loss: 0.2140 - accuracy: 0.9191
Epoch 17/24
31/31 [==============================] - 0s 1ms/step - loss: 0.2109 - accuracy: 0.9232
Epoch 18/24
31/31 [==============================] - 0s 1ms/step - loss: 0.2064 - accuracy: 0.9180
Epoch 19/24
31/31 [==============================] - 0s 987us/step - loss: 0.2023 - accuracy: 0.9232
Epoch 20/24
31/31 [==============================] - 0s 957us/step - loss: 0.1999 - accuracy: 0.9252
Epoch 21/24
31/31 [==============================] - 0s 1ms/step - loss: 0.1959 - accuracy: 0.9232
Epoch 22/24
31/31 [==============================] - 0s 1ms/step - loss: 0.1936 - accuracy: 0.9262
Epoch 23/24
31/31 [==============================] - 0s 1ms/step - loss: 0.1913 - accuracy: 0.9262
Epoch 24/24
8/8 [==============================] - 0s 772us/step - loss: 0.3158 - accuracy: 0.8811
Epoch 1/24
31/31 [==============================] - 0s 895us/step - loss: 0.5834 - accuracy: 0.6762
Epoch 2/24
31/31 

39/39 [==============================] - 0s 2ms/step - loss: 0.1966 - accuracy: 0.9287
Epoch 24/24
39/39 [==============================] - 0s 1ms/step - loss: 0.1948 - accuracy: 0.9303
(GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('imputer',
                                        SimpleImputer(strategy='median')),
                                       ('scaler', StandardScaler()),
                                       ('lr_model',
                                        LogisticRegression(penalty='elasticnet',
                                                           random_state=0,
                                                           solver='saga'))]),
             param_grid={'lr_model__C': [0.1, 1, 10],
                         'lr_model__l1_ratio': [0, 0.5, 1]}), GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('imputer',
                                        SimpleImputer(strategy='median')),
                                       ('scaler', St

In [10]:
def train_on_entire(X,y,pipe,grid_res):
    # fit pipeline
    pipe.set_params(**grid_res.best_params_).fit(X, y)
    # return the newly trained pipeline
    return pipe

In [11]:
train_on_entire(X_train,y_train,my_pipe1,my_grid1).score(X_test,y_test)

0.8950819672131147

In [12]:
train_on_entire(X_train,y_train,my_pipe2,my_grid2).score(X_test,y_test)

0.8885245901639345

In [13]:
train_on_entire(X_train,y_train,my_pipe3,my_grid3).score(X_test,y_test)

Epoch 1/24
39/39 [==============================] - 0s 2ms/step - loss: 0.6288 - accuracy: 0.6639
Epoch 2/24
39/39 [==============================] - 0s 2ms/step - loss: 0.5572 - accuracy: 0.7000
Epoch 3/24
39/39 [==============================] - 0s 2ms/step - loss: 0.4846 - accuracy: 0.7590
Epoch 4/24
39/39 [==============================] - 0s 2ms/step - loss: 0.4149 - accuracy: 0.8238
Epoch 5/24
39/39 [==============================] - 0s 2ms/step - loss: 0.3564 - accuracy: 0.8508
Epoch 6/24
39/39 [==============================] - 0s 1ms/step - loss: 0.3165 - accuracy: 0.8672
Epoch 7/24
39/39 [==============================] - 0s 1ms/step - loss: 0.2892 - accuracy: 0.8803
Epoch 8/24
39/39 [==============================] - 0s 1ms/step - loss: 0.2703 - accuracy: 0.8861
Epoch 9/24
39/39 [==============================] - 0s 1ms/step - loss: 0.2561 - accuracy: 0.8926
Epoch 10/24
39/39 [==============================] - 0s 2ms/step - loss: 0.2454 - accuracy: 0.8992
Epoch 11/24
39/39 [

0.8786885142326355